## Transfer Learning for Natural Language Modeling

構建一個情感分類器，根據評論的文字內容預測該評論是正面還是負面。
* 使用遷移學習 (從 wikitext-103 語言模型的預訓練權重開始)
* 調整這些權重以專注於 IMDB 電影評論的語言特性

### Language Models (語言模型)
語言模型（Language Model, LM） 是一種基於統計或深度學習的方法，用來理解和生成自然語言的工具。它的主要目標是 估算一段文字的語言結構和語意，並預測文字序列中下一個最可能出現的單詞或詞彙。語言模型在自然語言處理（NLP）中是核心技術，廣泛應用於機器翻譯、文本生成、語音識別和對話系統等領域。

語言模型的核心概念
* 機率分佈
* 條件機率

語言模型的類型
1. 統計語言模型：
   * 基於統計方法，例如 n-gram 模型。
   * 使用過去的文字序列來計算下一個單詞的機率。
   * 缺點：需要大量數據，且難以處理長距依賴。
2. 神經語言模型：
   * 基於深度學習，例如 RNN、LSTM、Transformer 等架構。
   * 能夠捕捉更複雜的語言結構。
   * 代表性模型：GPT、BERT、T5 等。

語言模型的應用
* 文本生成： 自動生成文章、故事、詩歌等內容。
* 機器翻譯： 將一種語言翻譯成另一種語言。
* 語音識別與輸入法： 幫助語音識別系統理解語句，或在輸入法中進行智能聯想和補全。
* 對話系統： 構建聊天機器人，能理解上下文並生成合理的回應。

研究科學家 Janelle Shane 在她的[部落格](https://aiweirdness.com/)和[推特](https://twitter.com/JanelleCShane)上分享了有關創意人工智慧的探索，主要涉及文字處理。
* [Why did the neural network cross the road?](https://aiweirdness.com/post/174691534037/why-did-the-neural-network-cross-the-road)
* [Try these neural network-generated recipes at your own risk.](https://aiweirdness.com/post/163878889437/try-these-neural-network-generated-recipes-at-your)
* [D&D character bios - now making slightly more sense](https://aiweirdness.com/post/183471928977/dd-character-bios-now-making-slightly-more)

### 斷詞（Tokenization）

斷詞是文本處理的第一步，目的是將原始句子分割成單詞，或更準確地說，是分割成「詞元（tokens）」。最簡單的方法是根據空格分割字符串，但我們可以採用更聰明的方法進行處理：
1. 處理標點符號： 標點符號需要特別處理，不能直接與單詞混在一起。
2. 處理縮寫詞： 一些單詞是由兩個不同單詞縮寫而成，例如 "isn't" 或 "don't"，需要將其拆分為 "is" 和 "n't"。
3. 清理文本內容： 如果文本中包含 HTML 代碼等不必要的內容，需要進行清理。

斷詞是自然語言處理中的關鍵步驟，通過智能化的處理方式，不僅能有效分割文本，還能處理標點符號、縮寫、清理雜訊，並引入特殊詞元來提升文本的結構化和可讀性。

### 數值化 ( Numericalization )

功能:
* Numericalization（數值化）是一種將文字資料轉換為數字形式的過程，主要用於自然語言處理（NLP）中的機器學習模型。
* 文字本身對機器無法直接理解，因此需要將文字轉換為數字表示，便於模型進行計算和分析。

步驟：
1. 分詞與詞元化（Tokenization）: 將文本拆分為更小的單位（例如單字、詞組或子詞），這些單位稱為「詞元」（tokens）。
2. 建立詞彙表（Vocabulary）: 創建一個詞彙表，包含文本中出現的所有詞元，並為每個詞元分配一個唯一的整數編碼。
3. 過濾與限制: 為了控制詞彙表的大小，可能會設定一些規則，例如只保留出現頻率超過一定次數的詞元，並用特殊詞元（如 "UNK" 表示未知詞元）替換那些未達標的詞元。
4. 轉換為數字序列: 將文本中的詞元替換為對應的整數編碼，從而將文字資料轉換為數字序列。

### 遷移學習 ( Transfer Learning )

Transfer Learning（遷移學習）是一種機器學習技術，指的是將在一個任務上訓練好的模型知識，應用到另一個相關任務中的方法。這種技術特別適合在資料有限或訓練資源不足的情況下使用，因為它能夠充分利用已經訓練好的模型，避免從頭開始訓練。

#### 藉由 Transfer Learning 建構 IMDb 語言模型
* pretrained weights from the `wikitext-103` language model.
* 利用該模型對英文的「知識」來構建分類器，但需要先對模型進行微調（fine-tuning），使其適應 IMDB 評論的資料集

#### WikiText-103 
* WikiText-103 是一個專為語言建模設計的資料集
* 內容來自 Wikipedia 中經過驗證的優質文章和特色文章，這些文章的品質較高，語言結構清晰且內容豐富。
* WikiText-103 包含超過 1 億個詞元（tokens），是 WikiText 系列中最大的版本。
* 資料集由完整的文章組成，而非單一的獨立句子或段落，這使得它特別適合用於需要處理長期依賴性的語言模型。
* 保留了語言的自然結構（如大小寫、標點符號等），有助於模型更準確地學習語言特性。

In [1]:
import torch

# 確保 GPU 可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


### 加載 IMDb資料集

In [2]:
from datasets import load_dataset

# load the IMDB dataset
dataset = load_dataset("imdb")
train_texts = dataset["train"]["text"]
test_texts = dataset["test"]["text"]
train_labels = dataset["train"]["label"]
test_labels = dataset["test"]["label"]

# 打印部分數據
print("訓練集樣本數:", len(train_texts))
print("測試集樣本數:", len(test_texts))
print("前3個訓練樣本:", train_texts[:3])
print("前3個訓練標籤:", train_labels[:3])

c:\anaconda\envs\py312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


訓練集樣本數: 25000
測試集樣本數: 25000
前3個訓練樣本: ['I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nud

### 數據預處理

將 IMDb 數據集轉換為模型可接受的格式。

In [ ]:
from sentence_transformers import InputExample
from torch.utils.data import DataLoader

# 將數據轉換為 Sentence Transformers 的 InputExample 格式
train_examples = [InputExample(texts=[text], label=float(label)) for text, label in zip(train_texts, train_labels)]
test_examples = [InputExample(texts=[text], label=float(label)) for text, label in zip(test_texts, test_labels)]


# 創建 DataLoader
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
test_dataloader = DataLoader(test_examples, shuffle=False, batch_size=16)

print("數據預處理完成，訓練集和測試集已轉換為可用格式。")

數據預處理完成，訓練集和測試集已轉換為可用格式。


### 微調語言模型

使用 Hugging Face 的 `Trainer` 進行模型的微調。

In [5]:
from sentence_transformers import SentenceTransformer, losses

# 加載 Sentence Transformer 模型
model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

# 定義損失函數
train_loss = losses.CosineSimilarityLoss(model)

# 訓練參數
num_epochs = 1
warmup_steps = int(len(train_dataloader) * num_epochs * 0.1)  # 10% 的 warmup

# 微調模型
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    output_path='model/finetuned_imdb_model'
)

print("模型微調完成，已保存至 './finetuned_imdb_model'")

IndexError: list index out of range

### 評估模型效能

In [ ]:
from sentence_transformers import evaluation

# 加載微調後的模型
finetuned_model = SentenceTransformer('./finetuned_imdb_model', device=device)

# 定義評估器
evaluator = evaluation.BinaryClassificationEvaluator(
    sentences1=test_texts[:1000],
    sentences2=test_texts[:1000],
    scores=[float(label) for label in test_labels[:1000]]
)

# 執行評估
evaluation_result = evaluator(finetuned_model)
print("模型效能評估結果:", evaluation_result)